In [1]:
import numpy as np
import math
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

#初始化
input_n = 4
hidden_n = 3
output_n = 3
v = np.random.random((input_n,hidden_n))  #输入层神经元与隐含层之间的权重（范围在0-1）
w = np.random.random((hidden_n,output_n)) #隐含层到输出层之间的权重
hidden_threshold = np.random.random(hidden_n) #隐含层阈值
output_threshold = np.random.random(output_n) #输出层阈值
hidden_values=[0]*hidden_n     #隐含层的输出值
output_values=[0]*output_n     #输出层的输出值
n = 0.1 #学习率

In [6]:
def label_to_value(label):
    #标签转换为对应输出值
    switch = {
        0.0: [1,0,0],
        1.0: [0,1,0],
        2.0: [0,0,1]
    }
    return switch[label]

def value_to_label(value):
    #神经网络输出值转换为对应标签
    return value.index(max(value)) 

def sigmoid(x):  
    return 1.0 / (1.0 + math.exp(-x))

def min_max_normalization(np_array):
    # 离差标准化，(Xi-min(X))/(max(X)-min(X))
    min_max_scaler = preprocessing.MinMaxScaler()  
    ret = min_max_scaler.fit_transform(np_array)  
    return ret

#计算当前样本输出
def predict(inputs,hidden_n,output_n):  
	#隐含层
	for i in range(hidden_n):
		total = 0.0
		for j in range(input_n):
			total += inputs[j]*v[j][i]
		hidden_values[i] = sigmoid(total-hidden_threshold[i])
	#输出层
	for i in range(output_n):
	 	total = 0.0
	 	for j in range(hidden_n):
	 		total += hidden_values[j]*w[j][i]
	 	output_values[i] = sigmoid(total-output_threshold[i])
	return output_values,hidden_values

#计算输出神经元的梯度项
def Gj(output_values,label):
	n = len(output_values)
	arr = np.array(label)
	gj = output_values*(np.ones(n)-output_values)*(arr-output_values)
	return gj

#计算隐层神经元的梯度项
def Eh(hidden_values,gj):
	wgj = []  # whjgj
	for i in range(hidden_n):
		s = 0.0
		for j in range(output_n):
			s += gj[j]*w[i][j]
		wgj.append(s)
	eh = hidden_values*(np.ones(hidden_n)-hidden_values)*wgj
	return eh

#更新权值和阈值
def Updata_wt(gj,eh,inputs):
	global v,w,output_threshold,hidden_threshold
	#输出层阈值更新
	output_threshold = output_threshold - n*gj

	#输入层和隐含层之间的权值更新
	for i in range(input_n):    
		for j in range(hidden_n):
			v[i][j] = v[i][j]+n*eh[j]*inputs[i]
	
	#隐含层阈值更新
	hidden_threshold = hidden_threshold - n*eh

	#隐含层到输出层之间的阈值更新
	for i in range(hidden_n):
		for j in range(output_n):
			w[i][j] = w[i][j] + n*gj[j]*hidden_values[i]

	#return v,w,hidden_threshold,output_threshold

def Network(Dataset,Labels):
	for x in range(1000): #训练轮数
		sample_num = len(Dataset)  #样本数量
		for i in range(sample_num):
			output_values,hidden_values = predict(Dataset[i],hidden_n,output_n) #计算当前样本的输出
			gj = Gj(output_values,Labels[i]) #计算输出神经元梯度项
			eh = Eh(hidden_values,gj)  #计算隐层神经元梯度项
			Updata_wt(gj,eh,Dataset[i]) #更新权值、阈值
	print("输入层到隐层之间的权值：\n",v)
	print("隐层到输出层之间的权值：\n",w)
	print("隐含层阈值：",hidden_threshold)
	print("输出层阈值：",output_threshold)

def LodeDataset():
	Dataset = [[0,0],
			   [1,0],
			   [1,1],
			   [0,2],
			   [2,2]]
	Labels = [1,-1,-1,-1,-1]
	return Dataset,Labels

#读取数据
def read_data(fileName):     
    numFeat = len(open(fileName).readline().split(' '))  #数据列数
    dataset = []; labels = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr =[]
        curLine = line.strip().split(' ')
        for i in range(2,numFeat):   #数据第一列为序号第二列为标签
            lineArr.append(float(curLine[i]))
        dataset.append(lineArr)
        labels.append(float(curLine[1]))
    return dataset,labels

#对标签进行转化
def Switch(Labels):
	switch_labels = [0]*len(Labels)
	for i in range(len(Labels)):
		switch_labels[i] = label_to_value(Labels[i])
	return switch_labels


Dataset,Labels = read_data('ButterflyFlower.txt')  #读取txt文件
switch_labels = Switch(Labels)  #标签转换

Xtrain,Xtest,Ytrain,Ytest = train_test_split(ret,switch_labels,test_size=0.3) #划分测试集和训练集
ret = min_max_normalization(Dataset)

print(Xtrain)
print(Ytrain)

[[1.         0.79661017 0.54166667 0.66666667]
 [0.08333333 0.05084746 0.625      0.19444444]
 [0.04166667 0.03389831 0.5        0.19444444]
 [0.95833333 0.77966102 0.58333333 0.55555556]
 [0.58333333 0.69491525 0.33333333 0.55555556]
 [0.         0.06779661 0.41666667 0.13888889]
 [0.04166667 0.03389831 0.83333333 0.41666667]
 [0.16666667 0.11864407 0.54166667 0.22222222]
 [0.375      0.52542373 0.29166667 0.41666667]
 [0.54166667 0.61016949 0.41666667 0.5       ]
 [0.79166667 0.6779661  0.20833333 0.38888889]
 [0.04166667 0.15254237 0.58333333 0.13888889]
 [0.5        0.71186441 0.41666667 0.66666667]
 [0.45833333 0.49152542 0.29166667 0.41666667]
 [0.83333333 0.94915254 0.41666667 0.91666667]
 [0.625      0.81355932 0.41666667 0.80555556]
 [0.91666667 0.69491525 0.45833333 0.72222222]
 [0.58333333 0.59322034 0.5        0.58333333]
 [0.375      0.38983051 0.16666667 0.16666667]
 [0.75       0.69491525 0.29166667 0.41666667]
 [0.79166667 0.66101695 0.33333333 0.36111111]
 [0.         

In [7]:
Network(Xtrain,Ytrain)  #训练神经网络

输入层到隐层之间的权值：
 [[ 4.15992266  1.02931013  6.05214722]
 [16.31470368  2.14223871  6.75078947]
 [-3.02704335 -1.21331115 -4.38844111]
 [-2.23611953  0.79479334  2.67106891]]
隐层到输出层之间的权值：
 [[ -5.72799939  11.64292714 -11.4888648 ]
 [ -1.84542197   1.3431706   -0.13044993]
 [ -8.24673818  -0.13346038  10.06244776]]
隐含层阈值： [10.97141321  1.72070388  2.31895787]
输出层阈值： [-4.8524268   6.26853758  4.40704776]


In [11]:
#计算准确率
def Test(Xtest,Ytest):
    n_true = 0
    n = len(Xtest)
    for i in range(n):
        o,h = predict(Xtest[i],hidden_n,output_n)
        pre = value_to_label(o) #转化为标签
        true_label = value_to_label(Ytest[i])
        if pre==true_label:
            n_true += 1
    accuracy = n_true / n
    print('正确率为：',accuracy)
Test(Xtest,Ytest)

正确率为： 0.9111111111111111
